In [ ]:
# Install required packages
# using ! to run linux shell commands
!pip install -q transformers clean-text[gpl] hazm

In [ ]:
# Import required packages
import json
import os
import copy
import collections

#### we have to ways to add or dataset very fast and easy 
1) using google drive 

2) using Kaggle


In [ ]:
# # # if your dataset is on google Drive use this BUt we use Kaggle now and i explain it in a bit
# from google.colab import drive
# drive.mount('/content/drive')

### Now what is [Kaggle](http://kaggle.com/) and what is our task

In [ ]:
#@title Kaggle Credential { display-mode: "form" }
import json
username = 'samantaghavisani' #@param {type: "string"}
api_key = '7961e9fb65a61a2f740ff6fc62ddc62b' #@param {type: "string"}


if username and api_key:
    token = {"username": username, "key": api_key}

    !mkdir ~/.kaggle
    !mkdir /content/.kaggle
    with open('/content/.kaggle/kaggle.json', 'w') as f:
        json.dump(token, f)

    !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json

    print('Your are ready to use kaggle API!')

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mkdir: cannot create directory ‘/content/.kaggle’: File exists
Your are ready to use kaggle API!


In [ ]:
#making a folder to hold our Dataset
!rm -rf /content/dataset
!mkdir -p /content/dataset
!kaggle datasets download -d saeedtqp/persian-digikala-reviwes

persian-digikala-reviwes.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/persian-digikala-reviwes.zip -d /content/dataset/

Archive:  /content/persian-digikala-reviwes.zip
  inflating: /content/dataset/2-p9vcb5bb.xlsx  


### Load the data using Pandas

In [ ]:

# numpy and pandas for np.arrays and datasets
import numpy as np
import pandas as pd

# Loading Digikala Dataset
data = pd.read_excel("/content/dataset/2-p9vcb5bb.xlsx")

#specify the comment type as str so to avoid errors
data.comment =data.comment.astype(str) 

# setting full display for pandas
# pd.set_option('display.max_colwidth', None)

In [ ]:
#viewing the data 
print(data.shape)
data.head()

(100000, 12)


,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages
0,3692,ماوس بی‌سیم لاجیتک مدل M325,IT,989472,0,0,verified,\N,NaN,واقعا عالیه. من که ازش خیلی راضیم,NaN,NaN
1,90213,شارژر همراه شیاومی مدل NDY-02-AN با ظرفیت 1000...,AC,3862150,4,1,verified,recommended,واقعاً عالیه,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,"[""عمر طولانی\r"",""افت بسیار کم میزان شارژ\r"",""ا...","[""ندارد""]"
2,59473,یدک پولیشر میکروفایبر مهسان مدل 20119,HW,626843,1,0,verified,not_recommended,خیلی سخت حوله اش در میاد,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,NaN,NaN
3,120499,گوشی موبایل هوآوی آنر مدل 5X KIW-L21 دو سیم‌کارت,MO,786887,6,11,verified,no_idea,گوشی مقرون به صرفه,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,"[""صفحه نمایش پرنور و با کیفیت\r"",""کیفیت بالای ...","[""کیفیت پایین اسپیکر\r"",""حاشیه خالی زیر صفحه ن..."
4,67200,شارژر همراه شیائومی مدل Mi ظرفیت 5000 میلی آمپ...,AC,854531,19,4,verified,no_idea,ابعاد، استحکام و نگهداری شارژ مناسب,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,"[""ظریف و زیبا و باریک\r"",""بدنه محکم و با دوام""]","[""ظرفیت متوسط""]"


In [ ]:
# dropping all other columns except comments and recommend
data.drop(['product_id','product_title','title_en','user_id' , 'verification_status','title','advantages','disadvantages','likes','dislikes'], axis=1, inplace=True) 

In [ ]:
#viewing the data Again
print(data.shape)
data.head()


(100000, 2)


,recommend,comment
0,\N,واقعا عالیه. من که ازش خیلی راضیم
1,recommended,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...
2,not_recommended,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...
3,no_idea,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...
4,no_idea,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...


In [ ]:
# taking care of some conflicts the easy way
data = data.dropna(subset=['recommend'])
data = data.dropna(subset=['comment'])
data = data.drop_duplicates(subset=['comment'], keep='first')
data = data.reset_index(drop=True)

In [ ]:
# previous information after solving the conflicts
# print data information
print(data.shape, '\n')

(96052, 2) 



### Normalization / Preprocessing




In [ ]:
# well you know what Hazm library does
#first we import it 
import hazm
# we also use regular expressions for some text cleaning steps
import re

In [ ]:

hazm.word_tokenize("غلط در جملات ! اشتباه است؟؟؟؟؟؟ نه 123 %&^%&68")

['غلط',
 'در',
 'جملات',
 '!',
 'اشتباه',
 'است',
 '؟؟؟؟؟؟',
 'نه',
 '123',
 '%&^%&',
 '68']

In [ ]:
# calculate the length of comments based on their words
data['comment_len_by_words'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))

In [ ]:
data.head(7)

,recommend,comment,comment_len_by_words
0,\N,واقعا عالیه. من که ازش خیلی راضیم,8
1,recommended,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,229
2,not_recommended,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,65
3,no_idea,همه چیز در رابطه با ظاهر این گوشی بسیار خوب اس...,273
4,no_idea,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,77
5,recommended,سلام دوستان،،\r\nمنم مثه بعضی از دوستان قبل از...,257
6,recommended,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,39


In [ ]:
# to draw simple illustrations and plots
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
#visualising to see the ideal sentence size
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=data['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.001,
    bargroupgap=0.001)

fig.show()

In [ ]:
# now we easily can find min and max for setting the sentence size
# but lets see the min and max 
min_max_len = data["comment_len_by_words"].min(), data["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 1 	Max: 1338


In [ ]:
#finding the ratio of the range we chose against the dataset
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [ ]:
data_gl_than(data, 128, 3)

Texts with word length of greater than 3 and less than 128 includes 94.50% of the whole!


In [ ]:
minlim, maxlim = 3, 128

In [ ]:
# remove comments with the length of fewer than three words or more than 128 length
data['comment_len_by_words'] = data['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['comment_len_by_words'])
data = data.reset_index(drop=True)

In [ ]:
# taking another glance at the data
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=data['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.01,
    bargroupgap=0.01)

fig.show()

### Text cleaning and Normalizing happens *here*

In [ ]:
# a great library to clean text and html files
from cleantext import clean

In [ ]:
sample_text ='مق1ال1ه WIRED  www.google.com/test و test@gmail.com به شماره ام 100 تا پیام امده تا حالا،:  +989173214   $$ '

In [ ]:
# using Cleantext lib to easily clean the text
sample_text_1 = clean(sample_text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=True,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="",
        replace_with_currency_symbol="",
    )
print(sample_text_1)

مق1ال1ه wired و به شماره ام 100 تا پیام امده تا حالا


In [ ]:
    # Hazm Again 
    # fun fact about parsbert: [unk] for roman digits 😉
    normalizer = hazm.Normalizer()
    
    sample_text = normalizer.normalize(sample_text)
    print(sample_text)

مق۱ال۱ه WIRED www. google. com/test و test@gmail. com به شماره‌ام ۱۰۰ تا پیام امده تا حالا،: +۹۸۹۱۷۳۲۱۴ $$ 


In [ ]:
sample_text_2 = "🤗 🍗واج🚲 "

In [ ]:
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    sample_text_2 = wierd_pattern.sub(r'', sample_text_2)
    print(sample_text_2)

 واج 


In [ ]:
# text cleaning and Normalizing all in one place
from cleantext import clean

def cleanhtml(raw_html):
    # simple html patterns
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    
    return cleantext


def cleaning(text):

    text = text.strip()
    
    # regular cleaning using clean text 
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

In [ ]:
cleaning(" غلط در جملات ! اشتباه است؟؟؟؟؟؟ 🤣 697748")

'غلط در جملات! اشتباه است؟؟؟؟؟؟ ۶۹۷۷۴۸'

In [ ]:
# cleaning comments
data['cleaned_comment'] = data['comment'].apply(cleaning)


# calculate the length of comments based on their words
data['cleaned_comment_len_by_words'] = data['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

# remove comments with the length of fewer than three words
data['cleaned_comment_len_by_words'] = data['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else len_t)
data = data.dropna(subset=['cleaned_comment_len_by_words'])
data = data.reset_index(drop=True)
# seeing the result of cleaning
data.head()

,recommend,comment,comment_len_by_words,cleaned_comment,cleaned_comment_len_by_words
0,\N,واقعا عالیه. من که ازش خیلی راضیم,8.0,واقعا عالیه. من که ازش خیلی راضیم,8
1,not_recommended,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,65.0,گیرههای فلزی خیلی سخت تا میشوند و لذا حوله را ...,57
2,no_idea,اگر ظرفیتش براتون کافیه حتما بخرید.\r\nیه شارژ...,77.0,اگر ظرفیتش براتون کافیه حتما بخرید. یه شارژر ۵...,73
3,recommended,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,39.0,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,38
4,\N,بوی تند ولی خوشبو داره.ماندگاریش خوبه و هر چقد...,16.0,بوی تند ولی خوشبو داره. ماندگاریش خوبه و هر چق...,16


In [ ]:
#dropping dirty comments
data = data[['cleaned_comment', 'recommend']]
data.columns = ['comment', 'recommend']
data.head()

,comment,recommend
0,واقعا عالیه. من که ازش خیلی راضیم,\N
1,گیرههای فلزی خیلی سخت تا میشوند و لذا حوله را ...,not_recommended
2,اگر ظرفیتش براتون کافیه حتما بخرید. یه شارژر ۵...,no_idea
3,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,recommended
4,بوی تند ولی خوشبو داره. ماندگاریش خوبه و هر چق...,\N


In [ ]:
fig = go.Figure()

groupby_label = data.groupby('recommend')['recommend'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
#Balancing the Data
negative_data = data[data['recommend'] == 'not_recommended']
positive_data = data[data['recommend'] == 'recommended']

cutting_point = min(len(negative_data), len(positive_data))

if cutting_point <= len(negative_data):
    negative_data = negative_data.sample(n=cutting_point).reset_index(drop=True)

if cutting_point <= len(positive_data):
    positive_data = positive_data.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([negative_data, positive_data])
new_data = new_data.sample(frac=1).reset_index(drop=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29884 entries, 0 to 29883
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comment    29884 non-null  object
 1   recommend  29884 non-null  object
dtypes: object(2)
memory usage: 467.1+ KB


In [ ]:
fig = go.Figure()

groupby_label = new_data.groupby('recommend')['recommend'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [NEW DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

## Train,Validation and Test 

In [ ]:
labels = list(sorted(new_data['recommend'].unique()))
print(labels)

['not_recommended', 'recommended']


In [ ]:
# train_test Easy spliting with sklearn
from sklearn.model_selection import train_test_split

In [ ]:
#setting a label id to represent the sentiments
new_data['recommend_id'] = new_data['recommend'].apply(lambda t: labels.index(t))

#splitting to tes, train and avlidation
train, test = train_test_split(new_data, test_size=0.1, random_state=1, stratify=new_data['recommend'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['recommend'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['comment'].values.tolist(), train['recommend_id'].values.tolist()
x_valid, y_valid = valid['comment'].values.tolist(), valid['recommend_id'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['recommend_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(24205, 3)
(2690, 3)
(2989, 3)


## TensorFlow

In [ ]:
import tensorflow as tf

### Configuration

In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3


#using ParsBert Uncased
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-saman-digikala/pytorch_model.bin'
# making a directory for outputing model
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
#linking sentiment labels to thier ids
labels = list(sorted(new_data['recommend'].unique()))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'not_recommended': 0, 'recommended': 1}
id2label: {0: 'not_recommended', 1: 'recommended'}


In [ ]:
from transformers import BertConfig, BertTokenizer
#using BERT Tokenizer 
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
#setting BERT config to realize the labels
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

# Printing the settings
print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "not_recommended",
    "1": "recommended"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "not_recommended": 0,
    "recommended": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.3.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



### Input Embeddings / Dataset

In [ ]:
from transformers import glue_convert_examples_to_features

from tqdm.notebook import tqdm

In [ ]:
#need more help here
class InputExample:
    """ A single example for simple sequence classification. """

    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


def make_examples(tokenizer, x, y=None, maxlen=128, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)

    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)
        
        if isinstance(_x, str):
            text_a = _x
            text_b = None
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    
    features = glue_convert_examples_to_features(
        examples, 
        tokenizer, 
        maxlen, 
        output_mode=output_mode, 
        label_list=list(np.unique(y)))

    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []

    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)

    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels

    return [xdata, ydata], features

In [ ]:
#toknizing and making TF dataset from test train and valid
train_dataset_base, train_examples = make_examples(tokenizer, x_train, y_train, maxlen=128)
valid_dataset_base, valid_examples = make_examples(tokenizer, x_valid, y_valid, maxlen=128)

test_dataset_base, test_examples = make_examples(tokenizer, x_test, y_test, maxlen=128)
[xtest, ytest], test_examples = make_examples(tokenizer, x_test, y_test, maxlen=128, is_tf_dataset=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:67: FutureWarning:

This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/text-classification/run_glue.py



In [ ]:
# what data we get after tokenizing
for value in train_dataset_base.take(1):
    print(f'     input_ids: {value[0]["input_ids"]}')
    print(f'attention_mask: {value[0]["attention_mask"]}')
    print(f'token_type_ids: {value[0]["token_type_ids"]}')
    print(f'        target: {value[1]}')

     input_ids: [    2  2844  5603 38993  5668 13171  2916  1379  2799  4589  2008  3924
  3949  2789  3346  5518  3171  4589 32778  4692 85715  2787 25251     4
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
attention_mask: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
# shufffling the datsets we need 
from sklearn.utils import shuffle

In [ ]:
#shuffling + batching the datasets 
def get_training_dataset(dataset, batch_size):
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
train_dataset = get_training_dataset(train_dataset_base, TRAIN_BATCH_SIZE)
valid_dataset = get_training_dataset(valid_dataset_base, VALID_BATCH_SIZE)

train_steps = len(train_examples) // TRAIN_BATCH_SIZE
valid_steps = len(valid_examples) // VALID_BATCH_SIZE

train_steps, valid_steps

(1512, 168)

### Model

In [ ]:
from transformers import TFBertModel, TFBertForSequenceClassification

In [ ]:
# a function to build the model 
def build_model(model_name, config, learning_rate=3e-5):
    #loading the model
    model = TFBertForSequenceClassification.from_pretrained(model_name, config=config)
    # setting the optimizer + loss function
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    #plus some basic metrics
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    # and compiling the model at the end
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    return model

In [ ]:
# building the model 
model = build_model(MODEL_NAME_OR_PATH, config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training

In [ ]:
#checking what kind of GPU we have 
!nvidia-smi

Fri Mar  5 11:10:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    28W /  70W |   8574MiB / 15109MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%time
#adding logs for tensorboard
import  datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# the start of training
r = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    epochs=EPOCHS,
    verbose=1, 
    callbacks=[tensorboard_callback])

final_accuracy = r.history['val_accuracy']
print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))

Epoch 1/3
1512/1512 [==============================] - 667s 434ms/step - loss: 0.2512 - accuracy: 0.8960 - val_loss: 0.1942 - val_accuracy: 0.9323
Epoch 2/3
1512/1512 [==============================] - 654s 432ms/step - loss: 0.1231 - accuracy: 0.9577 - val_loss: 0.2045 - val_accuracy: 0.9371
Epoch 3/3
1512/1512 [==============================] - 653s 432ms/step - loss: 0.0785 - accuracy: 0.9746 - val_loss: 0.2187 - val_accuracy: 0.9427
FINAL ACCURACY MEAN:  0.9373759826024374
CPU times: user 14min 55s, sys: 10min 24s, total: 25min 20s
Wall time: 33min 29s


In [ ]:
%
%tensorboard --logdir logs/fit

In [ ]:
# save the model we load it later to predict 

model.save_pretrained(os.path.dirname('/content/drive/MyDrive/NLP BERT/test2/'))

### Evaluation / Prediction

In [ ]:
# sklearn for reports
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:

evaluated = model.evaluate(test_dataset_base.batch(TEST_BATCH_SIZE))
print()
print(f'Evaluation: {evaluated}')
print()

predictions = model.predict(xtest)
ypred = predictions[0].argmax(axis=-1).tolist()

print()
print(classification_report(ytest, ypred, target_names=labels))
print()

print(f'F1: {f1_score(ytest, ypred, average="weighted")}')

###Load and predict




In [ ]:
#loading the saved model from previous step
trained_model = TFBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP BERT/test2/tf_model.h5', config=config)

Some layers from the model checkpoint at /content/drive/MyDrive/NLP BERT/test2/tf_model.h5 were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/NLP BERT/test2/tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
#initiating the pipline for prediction
from transformers import TextClassificationPipeline
say=TextClassificationPipeline(tokenizer=tokenizer,model=trained_model)

##### a random commnet form [Digikala](https://www.digikala.com/product/dkp-1492351/%D9%87%D9%86%D8%AF%D8%B2%D9%81%D8%B1%DB%8C-%D8%A8%D9%84%D9%88%D8%AA%D9%88%D8%AB-%D8%A7%D8%B1%D9%84%D8%AF%D8%A7%D9%85-%D9%85%D8%AF%D9%84-et-bh07#product-comment-list:~:text=%D8%AD%D8%AF%D9%88%D8%AF%20%DB%8C%DA%A9%20%D9%87%D9%81%D8%AA%D9%87%20%D9%87%D8%B3%D8%AA%20%DA%A9%D9%87%20%D8%A7%D8%B3%D8%AA%D9%81%D8%A7%D8%AF%D9%87,%DA%AF%D9%88%D8%B4%20%D9%87%D9%85%20%D8%AE%DB%8C%D9%84%DB%8C%20%D8%AE%D9%88%D8%A8%20%D9%85%DB%8C%20%D9%85%D9%88%D9%86%D9%87.)


In [ ]:

list_of_examples=[
                  'به مدت زیاد تو گوش بمونه اذیت میشه گوش موقع صحبت کردن با تلفن صدا خیلی ضعیفه وطرف مقابل وز وز میشنونه نمیدونم شاید برای من این طور هست ولی نظرات همه مثبته',
'حدود یک هفته هست که استفاده میکنم، در فروش ویژه خریدم و نسبت به قیمت واقعا راضی هستم. کیفیت صدای خوبی داره میکروفون خیلی خوب عمل میکنه روی گوش هم خیلی خوب می مون'
]

In [ ]:
#predicting
# say some examples !!!
say(list_of_examples)

###links to lookup
[Hugging face library](https://huggingface.co/)


[Optimization in Neural Networks](https://towardsdatascience.com/optimizers-for-training-neural-network-59450d71caf6#:~:text=Optimizers%20are%20algorithms%20or%20methods,order%20to%20reduce%20the%20losses.&text=Optimization%20algorithms%20or%20strategies%20are,the%20most%20accurate%20results%20possible.)

[what is a loss function](https://en.wikipedia.org/wiki/Loss_function)

[BERT](https://arxiv.org/abs/1810.04805)

[ParsBERT](https://arxiv.org/abs/2005.12515)